In [1]:
import numpy as np

In [2]:
'''
MIT License
Copyright (c) 2018 Fanjin Zeng
This work is licensed under the terms of the MIT license, see <https://opensource.org/licenses/MIT>.  
'''

def sliding_window_view(x, shape, step=None, subok=False, writeable=False):
    """
    Create sliding window views of the N dimensions array with the given window
    shape. Window slides across each dimension of `x` and provides subsets of `x`
    at any window position.
    Parameters
    ----------
    x : ndarray
        Array to create sliding window views.
    shape : sequence of int
        The shape of the window. Must have same length as number of input array dimensions.
    step: sequence of int, optional
        The steps of window shifts for each dimension on input array at a time.
        If given, must have same length as number of input array dimensions.
        Defaults to 1 on all dimensions.
    subok : bool, optional
        If True, then sub-classes will be passed-through, otherwise the returned
        array will be forced to be a base-class array (default).
    writeable : bool, optional
        If set to False, the returned array will always be readonly view.
        Otherwise it will return writable copies(see Notes).
    Returns
    -------
    view : ndarray
        Sliding window views (or copies) of `x`. view.shape = (x.shape - shape) // step + 1
    See also
    --------
    as_strided: Create a view into the array with the given shape and strides.
    broadcast_to: broadcast an array to a given shape.
    Notes
    -----
    ``sliding_window_view`` create sliding window views of the N dimensions array
    with the given window shape and its implementation based on ``as_strided``.
    Please note that if writeable set to False, the return is views, not copies
    of array. In this case, write operations could be unpredictable, so the return
    views is readonly. Bear in mind, return copies (writeable=True), could possibly
    take memory multiple amount of origin array, due to overlapping windows.
    For some cases, there may be more efficient approaches, such as FFT based algo discussed in #7753.
    Examples
    --------
    >>> i, j = np.ogrid[:3,:4]
    >>> x = 10*i + j
    >>> shape = (2,2)
    >>> sliding_window_view(x, shape)
    array([[[[ 0,  1],
             [10, 11]],
            [[ 1,  2],
             [11, 12]],
            [[ 2,  3],
             [12, 13]]],
           [[[10, 11],
             [20, 21]],
            [[11, 12],
             [21, 22]],
            [[12, 13],
             [22, 23]]]])
    >>> i, j = np.ogrid[:3,:4]
    >>> x = 10*i + j
    >>> shape = (2,2)
    >>> step = (1,2)
    >>> sliding_window_view(x, shape, step)
    array([[[[ 0,  1],
             [10, 11]],
            [[ 2,  3],
             [12, 13]]],
           [[[10, 11],
             [20, 21]],
            [[12, 13],
             [22, 23]]]])
    """
    # first convert input to array, possibly keeping subclass
    x = np.array(x, copy=False, subok=subok)

    try:
        shape = np.array(shape, np.int)
    except:
        raise TypeError('`shape` must be a sequence of integer')
    else:
        if shape.ndim > 1:
            raise ValueError('`shape` must be one-dimensional sequence of integer')
        if len(x.shape) != len(shape):
            raise ValueError("`shape` length doesn't match with input array dimensions")
        if np.any(shape <= 0):
            raise ValueError('`shape` cannot contain non-positive value')

    if step is None:
        step = np.ones(len(x.shape), np.intp)
    else:
        try:
            step = np.array(step, np.intp)
        except:
            raise TypeError('`step` must be a sequence of integer')
        else:
            if step.ndim > 1:
                raise ValueError('`step` must be one-dimensional sequence of integer')
            if len(x.shape)!= len(step):
                raise ValueError("`step` length doesn't match with input array dimensions")
            if np.any(step <= 0):
                raise ValueError('`step` cannot contain non-positive value')

    o = (np.array(x.shape)  - shape) // step + 1 # output shape
    if np.any(o <= 0):
        raise ValueError('window shape cannot larger than input array shape')

    strides = x.strides
    view_strides = strides * step

    view_shape = np.concatenate((o, shape), axis=0)
    view_strides = np.concatenate((view_strides, strides), axis=0)
    #view = np.lib.stride_tricks.as_strided(x, view_shape, view_strides, subok=subok, writeable=writeable)
    view = np.lib.stride_tricks.as_strided(x, view_shape, view_strides, subok=subok)#, writeable=writeable)

    if writeable:
        return view.copy()
    else:
        return view

In [3]:
i, j = np.ogrid[:9,:9]
arr = 9*i + j
arr

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23, 24, 25, 26],
       [27, 28, 29, 30, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49, 50, 51, 52, 53],
       [54, 55, 56, 57, 58, 59, 60, 61, 62],
       [63, 64, 65, 66, 67, 68, 69, 70, 71],
       [72, 73, 74, 75, 76, 77, 78, 79, 80]])

In [4]:
inner = 3
ghost = 2
outer = inner + 2*ghost

shape = (outer, outer)
steps = (inner, inner)

In [5]:
padding = np.pad(arr, [[ghost, ghost], [ghost, ghost]], mode='symmetric')
print(padding.shape)
print(padding)

(13, 13)
[[10  9  9 10 11 12 13 14 15 16 17 17 16]
 [ 1  0  0  1  2  3  4  5  6  7  8  8  7]
 [ 1  0  0  1  2  3  4  5  6  7  8  8  7]
 [10  9  9 10 11 12 13 14 15 16 17 17 16]
 [19 18 18 19 20 21 22 23 24 25 26 26 25]
 [28 27 27 28 29 30 31 32 33 34 35 35 34]
 [37 36 36 37 38 39 40 41 42 43 44 44 43]
 [46 45 45 46 47 48 49 50 51 52 53 53 52]
 [55 54 54 55 56 57 58 59 60 61 62 62 61]
 [64 63 63 64 65 66 67 68 69 70 71 71 70]
 [73 72 72 73 74 75 76 77 78 79 80 80 79]
 [73 72 72 73 74 75 76 77 78 79 80 80 79]
 [64 63 63 64 65 66 67 68 69 70 71 71 70]]


In [6]:
weights = np.zeros_like(padding)

In [7]:
v_padding = sliding_window_view(padding, shape, steps)
v_weights = sliding_window_view(weights, shape, steps)

In [8]:
print(padding.shape) # (num, num, outer, outer)

(13, 13)


In [9]:
print(padding[0, 0])

10


In [10]:
for y in range(v_padding.shape[0]):
    for x in range(v_padding.shape[1]):
        weight = np.ones_like(v_padding[y, x])
        v_weights[y, x] += weight
        

In [11]:
print(weights)

[[1 1 1 2 2 2 3 2 2 2 1 1 1]
 [1 1 1 2 2 2 3 2 2 2 1 1 1]
 [1 1 1 2 2 2 3 2 2 2 1 1 1]
 [2 2 2 4 4 4 6 4 4 4 2 2 2]
 [2 2 2 4 4 4 6 4 4 4 2 2 2]
 [2 2 2 4 4 4 6 4 4 4 2 2 2]
 [3 3 3 6 6 6 9 6 6 6 3 3 3]
 [2 2 2 4 4 4 6 4 4 4 2 2 2]
 [2 2 2 4 4 4 6 4 4 4 2 2 2]
 [2 2 2 4 4 4 6 4 4 4 2 2 2]
 [1 1 1 2 2 2 3 2 2 2 1 1 1]
 [1 1 1 2 2 2 3 2 2 2 1 1 1]
 [1 1 1 2 2 2 3 2 2 2 1 1 1]]


In [12]:
print(padding)

[[10  9  9 10 11 12 13 14 15 16 17 17 16]
 [ 1  0  0  1  2  3  4  5  6  7  8  8  7]
 [ 1  0  0  1  2  3  4  5  6  7  8  8  7]
 [10  9  9 10 11 12 13 14 15 16 17 17 16]
 [19 18 18 19 20 21 22 23 24 25 26 26 25]
 [28 27 27 28 29 30 31 32 33 34 35 35 34]
 [37 36 36 37 38 39 40 41 42 43 44 44 43]
 [46 45 45 46 47 48 49 50 51 52 53 53 52]
 [55 54 54 55 56 57 58 59 60 61 62 62 61]
 [64 63 63 64 65 66 67 68 69 70 71 71 70]
 [73 72 72 73 74 75 76 77 78 79 80 80 79]
 [73 72 72 73 74 75 76 77 78 79 80 80 79]
 [64 63 63 64 65 66 67 68 69 70 71 71 70]]


In [19]:
def invert(val):
    return 80-val

def weighted_map_blocks(arr, inner, ghost, func): # work for 3D, inner=[1, 3, 3], ghost=[0, 2, 2], 
    # param
    outer = inner + 2*ghost
    outer = [(i + 2*g) for i, g in zip(inner, ghost)]
    shape = outer
    steps = inner
        
    print(outer)
    print(shape)
    print(inner)
    # pad the array
    padding = np.pad(arr, [[ghost[0], ghost[0]], 
                           [ghost[1], ghost[1]], 
                           [ghost[2], ghost[2]]] , mode='symmetric')
    print(padding.shape)
    #print(padding)
    
    weights = np.zeros_like(padding)
    results = np.zeros_like(padding)
    
    v_padding = sliding_window_view(padding, shape, steps)
    v_weights = sliding_window_view(weights, shape, steps)
    v_results = sliding_window_view(results, shape, steps)
    
    for z in range(v_padding.shape[0]):
        for y in range(v_padding.shape[1]):
            for x in range(v_padding.shape[2]):
                # Get the result
                v_results[z,y,x] += func(v_padding[z,y,x]) ### Todo function is here
                
                v_weight = np.ones_like(v_padding[z,y,x])
                v_weights[z,y,x] += v_weight
                
    # Divided by the weight param
    results /= weights
    
    
    current_shape = results.shape
    trimmed_shape = [np.arange(ghost[0]),(results.shape[0] - ghost[0]), 
                     np.arange(ghost[1]),(results.shape[1] - ghost[1]), 
                     np.arange(ghost[2]),(results.shape[2] - ghost[2])]
    # Trim the result
    results = results[(ghost[0]):(results.shape[0] - ghost[0]), 
                      (ghost[1]):(results.shape[1] - ghost[1]), 
                      (ghost[2]):(results.shape[2] - ghost[2])]
    #results = results[trimmed_shape]
    
    return results

In [20]:
i, j = np.ogrid[:9,:9]
arr = 9*i + j
arr = np.expand_dims(arr, axis=0)
# print(arr.shape)
# print(arr)

inv = weighted_map_blocks(arr, [1, 3, 3], [0, 2, 2], invert)
print(inv)

[1, 7, 7]
[1, 7, 7]
[1, 3, 3]
(1, 13, 13)
[[[80 79 78 77 76 75 74 73 72]
  [71 70 69 68 67 66 65 64 63]
  [62 61 60 59 58 57 56 55 54]
  [53 52 51 50 49 48 47 46 45]
  [44 43 42 41 40 39 38 37 36]
  [35 34 33 32 31 30 29 28 27]
  [26 25 24 23 22 21 20 19 18]
  [17 16 15 14 13 12 11 10  9]
  [ 8  7  6  5  4  3  2  1  0]]]
